In [53]:
import numpy as np 
import pandas as pd 

dff = pd.read_csv('data_for_recommendation.csv')

In [58]:
print(len(user_id))
print(len(recipe_id))

1276
1734


In [2]:
user = pd.DataFrame(list(dff['user_id'].unique())).set_index(0)
user['viewed_recipes'] = dff.groupby('user_id').size()
user_id = list(user[user['viewed_recipes'] >= 20].index)

In [34]:
recipe = pd.DataFrame(list(dff['recipe_id'].unique())).set_index(0)
recipe['viewed_users'] = dff.groupby('recipe_id').size()
recipe_id = list(recipe[recipe['viewed_users'] >= 10].index)

In [35]:
cols = ['user_id', 'recipe_id', 'final_rating']
d1 = dff[dff['user_id'].isin(user_id)][cols]
d2 = d1[d1['recipe_id'].isin(recipe_id)][cols]

In [36]:
n_recipes = len(recipe_id)
n_users = len(user_id)
train_data_matrix = np.zeros((n_users, n_recipes))
print(train_data_matrix.shape)

sorted_user = sorted(user_id)
sorted_recipe = sorted(recipe_id)

(1276, 1734)


In [37]:
user_id_new = []
recipe_id_new = []
for i in range(0, len(d2)):
    user_id_new.append(sorted_user.index(d2['user_id'].iloc[i]))
    recipe_id_new.append(sorted_recipe.index(d2['recipe_id'].iloc[i]))

In [38]:
d2['user_id_new'] = user_id_new
d2['recipe_id_new'] = recipe_id_new

In [39]:
from sklearn import model_selection as cv
train_data, test_data = cv.train_test_split(d2, test_size = 0.2)

for line in train_data.itertuples():
    train_data_matrix[line[4] - 1, line[5] - 1] = line[3]

test_data_matrix = np.zeros((n_users, n_recipes))
for line in test_data.itertuples():
    test_data_matrix[line[4] - 1, line[5] - 1] = line[3]

In [40]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric = 'cosine')  
item_similarity = pairwise_distances(train_data_matrix.T, metric = 'cosine')

In [49]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1) # 每个用户的平均得分
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])    
    return pred

In [50]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [51]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()#nonzero(a)返回数组a中值不为零的元素的下标,相当于对稀疏矩阵进行提取
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

print ('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print ('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix))) 

User-based CF RMSE: 4.6688050294962675
Item-based CF RMSE: 4.682442506527549
